In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import math

In [76]:
trains = pd.read_csv('/content/drive/MyDrive/대기실/train.csv')
부산 = pd.read_csv('/content/drive/MyDrive/대기실/부산_날씨.csv')
창원 = pd.read_csv('/content/drive/MyDrive/대기실/창원_날씨.csv')
춘천 = pd.read_csv('/content/drive/MyDrive/대기실/춘천_날씨.csv')
경주 = pd.read_csv('/content/drive/MyDrive/대기실/경주_날씨.csv')
광주광역시 = pd.read_csv('/content/drive/MyDrive/대기실/광주광역시_날씨.csv')
장수 = pd.read_csv('/content/drive/MyDrive/대기실/장수_날씨.csv')
주유소 = pd.read_csv('/content/drive/MyDrive/대기실/주유소.csv', encoding='cp949')
소비자지수 = pd.read_csv('/content/drive/MyDrive/대기실/물가지수.csv')
수출입 = pd.read_csv('/content/drive/MyDrive/대기실/토마토_수출입.csv')
재배면적 = pd.read_csv('/content/drive/MyDrive/대기실/토마토 재배면적.csv')
실업률 = pd.read_csv('/content/drive/MyDrive/대기실/실업률_경제활동인구.csv')
최저시급 = pd.read_csv('/content/drive/MyDrive/대기실/최저시급.csv')

In [77]:
# 필요한 농산물 부분만 잘라쓰기
토마토 = trains[['date', '요일', '토마토_거래량(kg)', '토마토_가격(원/kg)']]
토마토

,date,요일,토마토_거래량(kg),토마토_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0
1,2016-01-02,토요일,30950.0,1621.0
2,2016-01-03,일요일,0.0,0.0
3,2016-01-04,월요일,291057.0,1834.0
4,2016-01-05,화요일,194626.5,1833.0
...,...,...,...,...
1728,2020-09-24,목요일,142487.0,5587.0
1729,2020-09-25,금요일,152575.5,5596.0
1730,2020-09-26,토요일,176807.0,5262.0
1731,2020-09-27,일요일,910.0,3781.0


In [78]:
# train.csv는 1733개 뿐. so 38개 row 만들어야 한다.
C = []
for x in range(38):
    if x % 7 == 0:
        C.append([0, '화요일', 0, 0])
    elif x % 7 == 1:
        C.append([0, '수요일', 0, 0])
    elif x % 7 == 2:
        C.append([0, '목요일', 0, 0])
    elif x % 7 == 3:
        C.append([0, '금요일', 0, 0])
    elif x % 7 == 4:
        C.append([0, '토요일', 0, 0])
    elif x % 7 == 5:
        C.append([0, '일요일', 0, 0])
    else:
        C.append([0, '월요일', 0, 0])

C = pd.DataFrame(C, columns = ['date', '요일', '토마토_거래량(kg)', '토마토_가격(원/kg)'])

# 직접 만든 C와 train.csv에서 뽑은 농산물을 합친다.
submission = pd.concat([토마토, C], axis=0)
submission

,date,요일,토마토_거래량(kg),토마토_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0
1,2016-01-02,토요일,30950.0,1621.0
2,2016-01-03,일요일,0.0,0.0
3,2016-01-04,월요일,291057.0,1834.0
4,2016-01-05,화요일,194626.5,1833.0
...,...,...,...,...
33,0,일요일,0.0,0.0
34,0,월요일,0.0,0.0
35,0,화요일,0.0,0.0
36,0,수요일,0.0,0.0


In [82]:
# 재배면적은 연간의 평균으로 이루어졌으므로 1771row를 만들기 위해 다음을 한다.
# 최저시급 : 연간으로 이루어져 1771row 만들어야 한다.
A = [];B = [];C = []
for x in range(5):
    if x == 0:
        for i in range(366):
            A.append(재배면적['토마토:면적(ha)'].iloc[x])
            B.append(재배면적['10a당 생산량(kg)'].iloc[x])
            C.append(최저시급['최저시급'].iloc[x])
    elif x < 4:
        for i in range(365):
            A.append(재배면적['토마토:면적(ha)'].iloc[x])
            B.append(재배면적['10a당 생산량(kg)'].iloc[x])
            C.append(최저시급['최저시급'].iloc[x])
    else:
        for i in range(310):
            A.append(재배면적['토마토:면적(ha)'].iloc[x])
            B.append(재배면적['10a당 생산량(kg)'].iloc[x])
            C.append(최저시급['최저시급'].iloc[x])

A = pd.DataFrame(A, columns = ['토마토:면적(ha)'])
B = pd.DataFrame(B, columns = ['10a당 생산량(kg)'])
C = pd.DataFrame(C, columns = ['최저시급'])

In [89]:
# 2020년 11월 초과한 게 있다면...
실업률 = 실업률[:59]

In [91]:
# 쉼표 is trap
for x in range(59):
    실업률['경제활동인구 (천명)'].iloc[x] = int(실업률['경제활동인구 (천명)'].iloc[x].replace(',', ''))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [92]:
# 실업률 : 월별 데이터이다. so 1771row를 만들기 위해 다음을 한다. 이 때 평균이 아니라면 해당 일 수로 나누자.
D = [];E = []
for x in range(59):
    # 29일까지 있는 2월(2016, 2020)
    if (x == 1) | (x == 49):
        for i in range(29):
            D.append(실업률['경제활동인구 (천명)'].iloc[x] / 29)
            E.append(실업률['실업률 (%)'].iloc[x])
    # 28일까지 있는 2월(2017 ~ 2019)
    elif x in [13, 25, 37]:
        for i in range(28):
            D.append(실업률['경제활동인구 (천명)'].iloc[x] / 28)
            E.append(실업률['실업률 (%)'].iloc[x])
    # 30일까지 있는 거
    elif x in [3,5,8,10, 15,17,20,22, 27,29,32,34, 39,41,44,46, 51,53,56]:
        for i in range(30):
            D.append(실업률['경제활동인구 (천명)'].iloc[x] / 30)
            E.append(실업률['실업률 (%)'].iloc[x])
    # 2020년 11월은 5일까지만 한다고 했다.
    elif x == 58:
        for i in range(5):
            D.append(실업률['경제활동인구 (천명)'].iloc[x] / 30)
            E.append(실업률['실업률 (%)'].iloc[x])
    # 31일까지 있는 거
    else:
        for i in range(31):
            D.append(실업률['경제활동인구 (천명)'].iloc[x] / 31)
            E.append(실업률['실업률 (%)'].iloc[x])

D = pd.DataFrame(D, columns = ['경제활동인구 (천명)'])
E = pd.DataFrame(E, columns = ['실업률 (%)'])

In [93]:
# 테스형! Unnamed: 0은 왜 생겨
부산 = 부산.drop('Unnamed: 0', axis=1)
창원 = 창원.drop('Unnamed: 0', axis=1)
춘천 = 춘천.drop('Unnamed: 0', axis=1)
경주 = 경주.drop('Unnamed: 0', axis=1)
광주광역시 = 광주광역시.drop('Unnamed: 0', axis=1)
장수 = 장수.drop('Unnamed: 0', axis=1)
소비자지수 = 소비자지수.drop('Unnamed: 0', axis=1)
수출입 = 수출입.drop('Unnamed: 0', axis=1)

# 지역 구분합니다.
부산.rename(columns = lambda x: "부산_" + x, inplace = True)
창원.rename(columns = lambda x: "창원_" + x, inplace = True)
춘천.rename(columns = lambda x: "춘천_" + x, inplace = True)
경주.rename(columns = lambda x: "경주_" + x, inplace = True)
광주광역시.rename(columns = lambda x: "광주광역시_" + x, inplace = True)
장수.rename(columns = lambda x: "장수_" + x, inplace = True)

In [94]:
submission = submission.reset_index()
중간단계 = pd.concat([submission, 소비자지수, 주유소[:1771], 부산, 창원, 춘천, 경주, 광주광역시, 장수], axis=1)

# 안 쓰는 날짜 column 지우기
중간단계 = 중간단계.drop(['index', '구분', '부산_날짜', '창원_날짜', '장수_날짜', '광주광역시_날짜', '경주_날짜', '춘천_날짜'], axis=1)

# 날짜 단일화
중간단계['date'] = 주유소['구분'][:1771]
중간단계

,date,요일,토마토_거래량(kg),토마토_가격(원/kg),지수,고급휘발유,보통휘발유,자동차용경유,실내등유,부산_평균기온,...,광주광역시_최저기온,광주광역시_평균운량,광주광역시_일강수량,광주광역시_일교차,장수_평균기온,장수_최고기온,장수_최저기온,장수_평균운량,장수_일강수량,장수_일교차
0,2016년01월01일,금요일,0.0,0.0,95.232,1780.99,1407.15,1188.71,835.38,5.3,...,-1.5,5.1,0.0,9.6,-2.8,5.9,-8.1,0.0,0.0,14.0
1,2016년01월02일,토요일,30950.0,1621.0,95.232,1781.69,1405.98,1187.91,834.11,8.1,...,1.3,4.8,0.0,11.4,2.8,10.2,-3.1,0.0,0.0,13.3
2,2016년01월03일,일요일,0.0,0.0,95.232,1780.84,1405.33,1187.10,833.42,11.4,...,3.7,4.0,0.0,10.0,4.3,15.5,-1.2,0.0,0.0,16.7
3,2016년01월04일,월요일,291057.0,1834.0,95.232,1780.73,1403.84,1185.85,832.00,10.4,...,2.3,3.1,0.0,9.1,3.4,9.9,-3.8,0.0,0.0,13.7
4,2016년01월05일,화요일,194626.5,1833.0,95.232,1779.60,1402.25,1184.01,830.39,5.3,...,-0.7,6.3,0.0,6.5,-1.8,2.9,-5.4,0.0,0.0,8.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766,2020년11월01일,일요일,0.0,0.0,100.090,1614.11,1324.32,1124.51,807.54,15.7,...,10.8,10.0,1.7,2.6,9.9,12.2,4.5,9.9,5.5,7.7
1767,2020년11월02일,월요일,0.0,0.0,100.090,1613.04,1323.37,1123.70,806.78,13.9,...,8.6,5.6,0.3,9.6,7.4,15.2,0.2,5.6,0.0,15.0
1768,2020년11월03일,화요일,0.0,0.0,100.090,1612.23,1322.14,1122.34,805.90,9.7,...,5.5,5.5,0.0,7.4,3.6,8.7,-0.6,3.4,0.0,9.3
1769,2020년11월04일,수요일,0.0,0.0,100.090,1610.77,1321.08,1121.22,805.10,8.7,...,4.4,1.4,0.0,8.5,2.0,10.2,-3.5,0.0,0.0,13.7


In [95]:
# 지역 비율 : 부산 33.6, 창원 17.5, 춘천 17, 경주 12.6, 광주광역시 11, 장수 8.3
final = 중간단계.copy()
final['fin평균기온'] = 중간단계['부산_평균기온'] * (336/1000) + 중간단계['창원_평균기온'] * (175/1000) + 중간단계['춘천_평균기온'] * (17/100) + 중간단계['경주_평균기온'] * (126/1000) + 중간단계['광주광역시_평균기온'] * (11/100) + 중간단계['장수_평균기온'] * (83/1000)
final['fin최고기온'] = 중간단계['부산_최고기온'] * (336/1000) + 중간단계['창원_최고기온'] * (175/1000) + 중간단계['춘천_최고기온'] * (17/100) + 중간단계['경주_최고기온'] * (126/1000) + 중간단계['광주광역시_최고기온'] * (11/100) + 중간단계['장수_최고기온'] * (83/1000)
final['fin최저기온'] = 중간단계['부산_최저기온'] * (336/1000) + 중간단계['창원_최저기온'] * (175/1000) + 중간단계['춘천_최저기온'] * (17/100) + 중간단계['경주_최저기온'] * (126/1000) + 중간단계['광주광역시_최저기온'] * (11/100) + 중간단계['장수_최저기온'] * (83/1000)
final['fin평균운량'] = 중간단계['부산_평균운량'] * (336/1000) + 중간단계['창원_평균운량'] * (175/1000) + 중간단계['춘천_평균운량'] * (17/100) + 중간단계['경주_평균운량'] * (126/1000) + 중간단계['광주광역시_평균운량'] * (11/100) + 중간단계['장수_평균운량'] * (83/1000)
final['fin일강수량'] = 중간단계['부산_일강수량'] * (336/1000) + 중간단계['창원_일강수량'] * (175/1000) + 중간단계['춘천_일강수량'] * (17/100) + 중간단계['경주_일강수량'] * (126/1000) + 중간단계['광주광역시_일강수량'] * (11/100) + 중간단계['장수_일강수량'] * (83/1000)
final['fin일교차'] = final['fin최고기온'] - final['fin최저기온']
final = final.drop(final.columns[9:45], axis=1)
hidfin = pd.concat([final, A, B, 수출입, C, D, E], axis=1)
hidfin

,date,요일,토마토_거래량(kg),토마토_가격(원/kg),지수,고급휘발유,보통휘발유,자동차용경유,실내등유,fin평균기온,...,fin일교차,토마토:면적(ha),10a당 생산량(kg),수출(중량),수출(금액),수입(중량),수입(금액),최저시급,경제활동인구 (천명),실업률 (%)
0,2016년01월01일,금요일,0.0,0.0,95.232,1780.99,1407.15,1188.71,835.38,2.2757,...,10.4473,6391,6107,10365.612903,25750.419355,134072.980645,149383.161290,6030,859.064516,3.7
1,2016년01월02일,토요일,30950.0,1621.0,95.232,1781.69,1405.98,1187.91,834.11,5.2459,...,9.8052,6391,6107,10365.612903,25750.419355,134072.980645,149383.161290,6030,859.064516,3.7
2,2016년01월03일,일요일,0.0,0.0,95.232,1780.84,1405.33,1187.10,833.42,7.0698,...,11.2334,6391,6107,10365.612903,25750.419355,134072.980645,149383.161290,6030,859.064516,3.7
3,2016년01월04일,월요일,291057.0,1834.0,95.232,1780.73,1403.84,1185.85,832.00,6.6233,...,9.5749,6391,6107,10365.612903,25750.419355,134072.980645,149383.161290,6030,859.064516,3.7
4,2016년01월05일,화요일,194626.5,1833.0,95.232,1779.60,1402.25,1184.01,830.39,2.0880,...,6.7985,6391,6107,10365.612903,25750.419355,134072.980645,149383.161290,6030,859.064516,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766,2020년11월01일,일요일,0.0,0.0,100.090,1614.11,1324.32,1124.51,807.54,13.1229,...,5.8539,5521,6231,19799.923333,48575.500000,128596.323333,150495.733333,8590,940.266667,3.4
1767,2020년11월02일,월요일,0.0,0.0,100.090,1613.04,1323.37,1123.70,806.78,12.1439,...,9.1793,5521,6231,19799.923333,48575.500000,128596.323333,150495.733333,8590,940.266667,3.4
1768,2020년11월03일,화요일,0.0,0.0,100.090,1612.23,1322.14,1122.34,805.90,7.6596,...,8.6505,5521,6231,19799.923333,48575.500000,128596.323333,150495.733333,8590,940.266667,3.4
1769,2020년11월04일,수요일,0.0,0.0,100.090,1610.77,1321.08,1121.22,805.10,6.1720,...,12.0067,5521,6231,19799.923333,48575.500000,128596.323333,150495.733333,8590,940.266667,3.4


In [96]:
hidfin.to_csv('토마토이다.csv')